In [2]:
import pandas as pd
import numpy as np
import requests

Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [4]:
base_url = 'https://python.zach.lol'
items_url = 'https://python.zach.lol/api/v1/items'
sales_url = 'https://python.zach.lol/api/v1/sales'
stores_url = 'https://python.zach.lol/api/v1/stores'

def get_data(key, request_url, base_url):
    response = requests.get(request_url)
    data = response.json()
    page = data['payload']['page']
    max_page = data['payload']['max_page']
    df = pd.DataFrame(data['payload'][key])
    while page <= max_page:
        next_page = data['payload']['next_page']
        if next_page is None or next_page == "None":
            break
        response = requests.get(base_url + next_page)
        data = response.json()
        df = pd.concat([df, pd.DataFrame(data['payload'][key])])
        page = data['payload']['page']

    df.reset_index(inplace=True)
    return df

sales = get_data('sales', sales_url, base_url)
sales.shape

(913000, 6)

In [ ]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

In [ ]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

In [ ]:
data['payload']

In [ ]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

In [ ]:
df = pd.DataFrame(data['payload']['sales'])
df.shape

Do the same thing, but for stores.

In [ ]:
print(data['payload']['next_page'])

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/items')
data = response.json()
page = data['payload']['page']
max_page = data['payload']['max_page']
while page <= max_page:
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()
    page = data['payload']['page']
    


Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

Save the data in your files to local csv files so that it will be faster to access in the future.

Combine the data from your three separate dataframes into one large dataframe.

Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.